# Projeto 1 - Ciência dos Dados

Nome: Enzo Junqueira

Nome: Thomas Rudge

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [83]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [84]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Enzo\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documentos\INSPER\2023\2-SEMESTRE\CIENCIA-DOS-DADOS\Projeto 1\Projeto1_CDADOS


Carregando a base de dados com os tweets classificados manualmente:

In [85]:
treino = pd.read_excel('dados_treino.xlsx')
treino.head()

,Mensagem,"Target: Avaliaremos quais reviews estao relacionados à utilização do Kindle, conteúdo do livro ou com erros de edição do livro (1 = Conteúdo, 2 = Amazon , 3 = Edição)",Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Comprei o livro e ele não baixou. Gostaria de ...,2,NaN,Rótulos de Linha,Contagem de Mensagem
1,"Muita enrolação, pouca informação. A autora fi...",1,NaN,1,190
2,"Não gostei, a capa diz mais do que o conteúdo ...",1,NaN,2,72
3,Estava tentando há dias continuar lendo sem qu...,1,NaN,3,38
4,Incapaz de conviver com ideias diferentes na a...,1,NaN,Total Geral,300


In [86]:
teste = pd.read_excel('dados_teste.xlsx')
teste.head()

,Mensagem,"Target: Avaliaremos quais reviews estao relacionados à utilização do Kindle, conteúdo do livro ou com erros de edição do livro (1 = Conteúdo, 2 = Amazon , 3 = Edição)",Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Como o livro se concentra na política american...,1,NaN,Rótulos de Linha,Contagem de Mensagem
1,"Boa estória. Quis ler antes de ver o filme, qu...",3,NaN,1,131
2,Não gostei. O livro o tempo todo traz narrativ...,1,NaN,2,53
3,Muito confuso.,1,NaN,3,16
4,"Comprei o livro logo após comprar ""Garota Exem...",1,NaN,(vazio),NaN


Renomearemos a coluna de rótulos para algo mais simples, como "Target".
Descartaremos as colunas desnecessárias.
Carregaremos a lista de stop words do arquivo fornecido.

In [87]:
treino.columns = ['Mensagem', 'Target', 'Drop1', 'Drop2', 'Drop3']
teste.columns = ['Mensagem', 'Target', 'Drop1', 'Drop2', 'Drop3']

treino = treino[['Mensagem', 'Target']]
teste = teste[['Mensagem', 'Target']]

In [95]:

from unidecode import unidecode

with open('stopwords.txt', 'r') as file:
    stopwords = file.read().splitlines()

stopwords = [unidecode(word.lower()) for word in stopwords]

# Splitting the single string in the stopwords_list into individual words
stopwords = stopwords[0].split(', ')


Limpar as mensagens, removendo caracteres especiais, números e pontuações.
Converter todas as palavras para minúsculas para manter a uniformidade.
Remover as stop words.

In [96]:

import re
from unidecode import unidecode

def limpa_texto(text, stopwords):
    
     # Remove acentos e caracteres especiais usando unidecode
    novo_texto = unidecode(text)

   
    # Remove todos os caracteres que não são letras ou números
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', novo_texto)

    text = cleaned_text.lower()
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text


# Aplica a função limpa_texto para todas as mensagens
treino['Mensagem'] = treino['Mensagem'].apply(lambda x: limpa_texto(x, stopwords))
teste['Mensagem'] = teste['Mensagem'].apply(lambda x: limpa_texto(x, stopwords))


treino.head(), teste.head()


(                                            Mensagem  Target
 0  comprei baixou gostaria saber vai baixar quero...       2
 1  enrolacao informacao autora fica contando vida...       1
 2                          gostei capa mais conteudo       1
 3  tentando dias continuar lendo ficasse sono inc...       1
 4  incapaz conviver ideias diferentes academia br...       1,
                                             Mensagem  Target
 0  concentra politica americana duas decadas espe...       1
 1  boa estoria quis ler filme epoca fez sucesso e...       3
 2  gostei tempo traz narrativas cenarios eua bols...       1
 3                                            confuso       1
 4  comprei logo comprar garota exemplar sair qual...       1)

___
## Classificador automático


Target: Avaliaremos quais reviews estão relacionados à utilização do Kindle, conteúdo do livro ou com erros de edição do livro

1. Kindle: Reviews relacionados à experiência de leitura no dispositivo Kindle.
2. Conteúdo: Reviews que discutem o conteúdo do livro, como enredo, personagens, estilo de escrita, etc.
3. Erros de Edição: Reviews que mencionam erros de edição, como problemas de formatação, erros de impressão...


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

1. Calcular as probabilidades a priori para cada classe (a frequência de cada classe nos dados de treinamento).
2. Calcular as probabilidades condicionais de cada palavra dada a classe (a frequência de cada palavra em cada classe nos dados de treinamento).
3. Aplicar a suavização de Laplace para lidar com palavras não vistas nos dados de treinamento.

In [114]:
# Calcula a probabilidade de cada classe
prob = treino['Target'].value_counts(normalize=True)
prob

1    0.633333
2    0.240000
3    0.126667
Name: Target, dtype: float64

As probabilidades a priori para cada classe (Target) são as seguintes:

1. Conteúdo: 63.33%
2. Kindle: 24.00%
3. Erros de Edição: 12.67%

In [105]:
# Conteúdo
conteudo = treino[treino.Target == 1]
# conteudo.head()

In [106]:
# kindle
kindle = treino[treino.Target == 2]
# kindle.head()

In [107]:
# edição
edicao = treino[treino.Target == 3]
# edicao.head()

In [102]:
from collections import Counter

# Defining a function to count words in a DataFrame
def count_words(df):
    # Concatenating all the messages and splitting into words
    words = ' '.join(df['Mensagem']).split()
    # Counting the frequency of each word
    word_count = Counter(words)
    return word_count

# Counting the words in each subcategory
conteudo_word_count = count_words(conteudo)
kindle_word_count = count_words(kindle)
edicao_word_count = count_words(edicao)

total_word_count = conteudo_word_count + kindle_word_count + edicao_word_count



# Displaying the most common words in each subcategory
conteudo_word_count.most_common(5), kindle_word_count.most_common(5), edicao_word_count.most_common(5)



([('mais', 85), ('ler', 47), ('historia', 46), ('ser', 44), ('autor', 43)],
 [('amazon', 34),
  ('mais', 32),
  ('preco', 25),
  ('kindle', 22),
  ('produto', 21)],
 [('mais', 16), ('veio', 15), ('box', 15), ('paginas', 14), ('produto', 10)])

In [118]:
# Calculating the prior probabilities for each category
prior_prob_conteudo = len(conteudo) / len(treino)
prior_prob_kindle = len(kindle) / len(treino)
prior_prob_edicao = len(edicao) / len(treino)

# Calculating the total number of words in each subcategory
total_words_conteudo = sum(conteudo_word_count.values())
total_words_kindle = sum(kindle_word_count.values())
total_words_edicao = sum(edicao_word_count.values())

# Calculating the total number of unique words in the entire training set
total_unique_words = len(set(total_word_count.keys()))

# Displaying the prior probabilities and total number of words
prior_prob_conteudo, prior_prob_kindle, prior_prob_edicao, total_words_conteudo, total_words_kindle, total_words_edicao, total_unique_words

print('Probabilidade de estar em conteudo: ', prior_prob_conteudo)
print('Probabilidade de estar em kindle: ', prior_prob_kindle)
print('Probabilidade de estar em edicao: ', prior_prob_edicao)
print('Total de palavras em conteudo: ', total_words_conteudo)
print('Total de palavras em kindle: ', total_words_kindle)
print('Total de palavras em edicao: ', total_words_edicao)
print('Total de palavras unicas: ', total_unique_words)



Probabilidade de estar em conteudo:  0.6333333333333333
Probabilidade de estar em kindle:  0.24
Probabilidade de estar em edicao:  0.12666666666666668
Total de palavras em conteudo:  4970
Total de palavras em kindle:  1859
Total de palavras em edicao:  918
Total de palavras unicas:  3380


In [119]:
def calculate_conditional_probabilities(word_count, total_words, total_unique_words):
    # Suavização de Laplace
    probabilities = {}
    for word, count in word_count.items():
        probabilities[word] = (count + 1) / (total_words + total_unique_words)
    return probabilities


# Calculating the conditional probabilities for each word given the category
prob_conteudo = calculate_conditional_probabilities(conteudo_word_count, total_words_conteudo, total_unique_words)
prob_kindle = calculate_conditional_probabilities(kindle_word_count, total_words_kindle, total_unique_words)
prob_edicao = calculate_conditional_probabilities(edicao_word_count, total_words_edicao, total_unique_words)

# Displaying the conditional probabilities of the first few words in the 'conteudo' category
list(prob_conteudo.items())[:5]

[('enrolacao', 0.00047904191616766467),
 ('informacao', 0.00023952095808383233),
 ('autora', 0.0019161676646706587),
 ('fica', 0.0008383233532934132),
 ('contando', 0.00023952095808383233)]

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

### Por que não usar o próprio classificador para gerar mais amostras de treinamento?
Usar o próprio classificador para gerar mais amostras de treinamento pode levar a problemas de viés e superajuste (overfitting). O modelo já foi treinado com o conjunto de dados existente e tem suas limitações e viéses baseados nesse conjunto de dados. Adicionar mais amostras geradas pelo próprio modelo só reforçará os viéses e limitações existentes, e não contribuirá para a generalização do modelo para dados não vistos. Além disso, pode levar a uma falsa sensação de precisão e confiança no modelo, já que as amostras adicionadas são, por definição, bem classificadas pelo modelo.

### Diferentes cenários para Naïve Bayes
1. Diagnóstico Médico:
O classificador Naïve Bayes pode ser usado para diagnosticar doenças com base em sintomas. Cada sintoma pode ser tratado como uma característica, e o Naïve Bayes pode ser usado para calcular a probabilidade de uma doença específica dado um conjunto de sintomas.

2. Análise de Sentimento em Redes Sociais:
O Naïve Bayes pode ser empregado para analisar o sentimento de usuários em redes sociais, classificando postagens ou comentários como positivos, negativos ou neutros. Isso pode ser útil para empresas monitorarem a opinião pública sobre seus produtos ou serviços e ajustarem suas estratégias de marketing e comunicação conforme necessário.

### Sugestões de Melhorias e Como Implementar:
1. Limpeza e Pré-processamento Avançado:
Como Implementar: Utilizar técnicas mais avançadas de processamento de linguagem natural, como a lematização, para reduzir as palavras à sua forma base e remover variações.
Referências: Bibliotecas como NLTK e SpaCy em Python oferecem ferramentas avançadas de pré-processamento de texto.
Link: https://codewithgolu.com/python/a-comprehensive-guide-to-text-preprocessing-with-nltk-and-spa-cy/

2. Balanceamento de Classes:
Como Implementar: Utilizar técnicas de reamostragem como oversampling ou undersampling para equilibrar a distribuição das classes no conjunto de treinamento.
Referências: A biblioteca imbalanced-learn em Python oferece métodos para lidar com conjuntos de dados desbalanceados. Link: https://www.analyticsvidhya.com/blog/2022/05/handling-imbalanced-data-with-imbalance-learn-in-python/

3. Otimização de Hiperparâmetros:
Como Implementar: Utilizar técnicas de otimização de hiperparâmetros, como busca em grade (Grid Search), para encontrar os melhores parâmetros para o modelo.
Referências: A biblioteca Scikit-learn em Python oferece métodos como GridSearchCV para otimização de hiperparâmetros. Link: https://pyimagesearch.com/2021/05/24/grid-search-hyperparameter-tuning-with-scikit-learn-gridsearchcv/

4. Seleção de Características:
Como Implementar: Empregar técnicas de seleção de características para identificar e manter apenas as características mais informativas e remover características redundantes ou irrelevantes.
Referências: Métodos como o teste do qui-quadrado (chi-squared test) podem ser usados para avaliar a importância das características.

5. Uso de Modelos Mais Complexos:
Como Implementar: Experimentar modelos mais avançados e complexos como Máquinas de Vetores de Suporte (SVM) ou Redes Neurais.
Referências: Frameworks como TensorFlow e PyTorch permitem a implementação de modelos mais avançados de aprendizado de máquina.


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**